In [1]:
import mysql.connector
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
db = mysql.connector.connect(
    host= 'iosqlde.onairnet.xyz',
    user= 'vienna_airport',
    password= 'Vie23@#@',
    database= 'aeropuerto_viena',
)

In [3]:
def sql(_query):
    return pd.read_sql_query(_query, db)

1. ¿Cuantos litros se han repostado en cada zona operacional? Ordénalo de menor a mayor


In [11]:
sql('''SELECT count(id_gasolinera) as cantidad,sum(litros) as litros_totales,
CASE
	WHEN id_gasolinera=3 then 'ZONA B'
    WHEN id_gasolinera=6 then 'ZONA D'
    WHEN id_gasolinera=1 THEN 'ZONA A'
    WHEN id_gasolinera=2 THEN 'ZONA A'
    WHEN id_gasolinera=5 THEN 'ZONA C'
    WHEN id_gasolinera=4 THEN 'ZONA C'
    WHEN id_gasolinera=9 THEN 'ZONA E'
    WHEN id_gasolinera=7 THEN 'ZONA E'
    WHEN id_gasolinera=8 THEN 'ZONA E'
END AS zona_oper
FROM aeropuerto_viena.repostajes
group by zona_oper
order by litros_totales asc''')

,cantidad,litros_totales,zona_oper
0,378,10094.319983,ZONA E
1,406,10116.049991,ZONA C
2,392,10340.810000,ZONA D
3,411,10392.599996,ZONA B
4,416,10892.939974,ZONA A


2. ¿Cuántos vehículos han repostado más de 5L de RyanairHandling en la zona operacional A?¿Y en la E?


In [12]:
sql('''SELECT zonas_operacionales.nombre as Zonas,count(id_gasolinera) as cantidad,sum(repostajes.litros) as 
total_litros,handling.nombre as Operador
from vehiculos
inner join handling on id_handling=handling.id
inner join repostajes on id_vehiculo=vehiculos.id
inner join zonas_operacionales on vehiculos.id_zona=zonas_operacionales.id
where litros>5 and handling.nombre ='RYANAIR HANDLING' AND (zonas_operacionales.nombre LIKE '%A' OR zonas_operacionales.nombre LIKE '%E')
group by zonas_operacionales.nombre
order by total_litros desc''')

,Zonas,cantidad,total_litros,Operador
0,ZONA E,92,2484.470005,RYANAIR HANDLING
1,ZONA A,76,2203.579996,RYANAIR HANDLING


3. ¿Cuántos vehículos de media dan servicio a cada vuelo?


In [13]:
sql('''select total_vuelos,total as cantidad_veh,round(total/total_vuelos,2) as promedio_serv from(
select count(id_vuelo) as total_vuelos,sum(cantidad) as total from(
select id_vuelo,count(id_vehiculo) as cantidad FROM aeropuerto_viena.servicios
group by id_vuelo)sc)sc2''')

,total_vuelos,cantidad_veh,promedio_serv
0,3003,11968.0,3.99


4. ¿Cuál es el mayor operador de handling cada mes por cantidad de minutos de servicio prestados?


In [6]:
sql('''Select nombre,min_totales,
case
	when Mes=1 then'Enero'
    when Mes=2 then'Febrero'
    when Mes=3 then'Marzo'
    when Mes=4 then'Abril'
    when Mes=5 then'Mayo'
    when Mes=6 then'Junio'
    when Mes=7 then'Julio'
    when Mes=8 then'Agosto'
    when Mes=9 then'Septiembre'
    when Mes=10 then'Octubre'
    when Mes=11 then'Noviembre'
    when Mes=12 then'Diciembre'
end as mes
from(
select handling.nombre,
sum(minutos) as min_totales, 
month(vuelos.fecha) AS Mes FROM aeropuerto_viena.servicios
inner join aeropuerto_viena.vehiculos on id_vehiculo=vehiculos.id
inner join aeropuerto_viena.handling on vehiculos.id_handling=handling.id
inner join aeropuerto_viena.vuelos on servicios.id_vuelo=vuelos.id
group by handling.nombre,month(vuelos.fecha)
order by Mes,min_totales desc) sc LIMIT 5''')

,nombre,min_totales,mes
0,VIENNA HANDLING,8022.0,Enero
1,RYANAIR HANDLING,7311.0,Enero
2,EASYHANDLING,6429.0,Enero
3,GROUND FORCE,5738.0,Enero
4,SWISSPORT,5273.0,Enero


5. Teniendo en cuenta que cada hora de servicio se factura a 60€, ¿Cuál es el beneficio por vehículo por cada empresa de handling?(Ten en cuenta que el combustible es un coste para ese vehículo)

In [7]:
sql('''SELECT sc3.nombre, sc3.tipo, round(sc3.beneficio - sc2.prec_repost_total,2) as beneficio_sin_repos
FROM (
    SELECT nombre, min_totales, tipo, ROUND((min_totales/60)*60,0) as beneficio
    FROM (
        SELECT handling.nombre, SUM(minutos) as min_totales, vehiculos.tipo
        FROM aeropuerto_viena.servicios
        INNER JOIN aeropuerto_viena.vehiculos ON id_vehiculo=vehiculos.id
        INNER JOIN aeropuerto_viena.handling ON vehiculos.id_handling=handling.id
        INNER JOIN aeropuerto_viena.vuelos ON servicios.id_vuelo=vuelos.id
        GROUP BY handling.nombre, tipo
        ORDER BY nombre
    ) sc
) sc3
LEFT JOIN (
    SELECT nombre, tipo, ROUND(SUM(litros*precio_litro),2) as prec_repost_total
    FROM aeropuerto_viena.repostajes
    INNER JOIN aeropuerto_viena.vehiculos ON repostajes.id_vehiculo=vehiculos.id
    INNER JOIN aeropuerto_viena.handling ON vehiculos.id_handling=handling.id
    GROUP BY nombre, tipo
    ORDER BY nombre
) sc2 ON sc3.nombre = sc2.nombre AND sc3.tipo = sc2.tipo LIMIT 5''')

,nombre,tipo,beneficio_sin_repos
0,EASYHANDLING,Ambulift,5870.62
1,EASYHANDLING,Autobus Lanzadera,9964.61
2,EASYHANDLING,Camión de repostaje,4796.45
3,EASYHANDLING,Cinta Maletas,9814.11
4,EASYHANDLING,Coche de servicio,12736.70


6. ¿Cuánto tiempo está ocupada cada posición de media en cada zona operacional?

In [17]:
sql('''SELECT posiciones.zone_id AS Zona_Operacional, posiciones.nombre AS Posicion, ROUND(AVG(servicios.minutos), 2) AS Media_Ocupacion,
        CASE
	WHEN zone_id=1 then 'ZONA A'
    WHEN zone_id=2 then 'ZONA B'
    WHEN zone_id=3 THEN 'ZONA C'
    WHEN zone_id=4 THEN 'ZONA D'
    WHEN zone_id=5 THEN 'ZONA E' END AS Zona_Oper
FROM posiciones 
JOIN vuelos ON posiciones.id = vuelos.id_posicion 
JOIN servicios ON vuelos.id = servicios.id_vuelo 
GROUP BY posiciones.zone_id, posiciones.nombre
ORDER BY posiciones.zone_id DESC''')
#REVISAR

,Zona_Operacional,Posicion,Media_Ocupacion,Zona_Oper
0,5,A144,32.21,ZONA E
1,5,A38,31.54,ZONA E
2,5,B71,32.88,ZONA E
3,5,D105,32.65,ZONA E
4,5,E109,32.23,ZONA E
...,...,...,...,...
177,1,X106,39.08,ZONA A
178,1,Y86,28.27,ZONA A
179,1,Z141,33.45,ZONA A
180,1,Z22,31.02,ZONA A


7. ¿Cuántas posiciones tiene cada zona operacional?


In [14]:
#Opción 2
sql('''SELECT zone_id, COUNT(DISTINCT ID) as Posiciones_Zona_OPS,    
    CASE
	WHEN zone_id=1 then 'ZONA A'
    WHEN zone_id=2 then 'ZONA B'
    WHEN zone_id=3 THEN 'ZONA C'
    WHEN zone_id=4 THEN 'ZONA D'
    WHEN zone_id=5 THEN 'ZONA E' END AS Zona_Oper FROM posiciones GROUP BY zone_id''')
#Sumamos el total de las posiciones por cada zona operacional usando un DISTINCT en ID de cada zone_id, finalmente agrupamos por zone_id para obtener la tabla.

,zone_id,Posiciones_Zona_OPS,Zona_Oper
0,1,78,ZONA A
1,2,71,ZONA B
2,3,63,ZONA C
3,4,64,ZONA D
4,5,74,ZONA E


8. ¿Cuántas posiciones ha servido cada vehículo?


In [19]:
sql('''SELECT vehiculos.tipo AS Tipo_Vehiculo, 
    COUNT(posiciones.id) AS Posiciones_Servidas FROM vehiculos 
    JOIN zonas_operacionales ON vehiculos.id_zona = zonas_operacionales.id 
    JOIN posiciones ON zonas_operacionales.id = posiciones.zone_id GROUP BY vehiculos.tipo''')
#Se hace un Join de 3 tablas que utilizan el mismo ID (zonas_operacionales: ID, vehiculos: id_zona y posiciones: zone:id)
#En esta tabla se selecciona la columna tipo de la tabla Vehiculos, se renombra y se hace un count del ID de la tabla Posiciones, se renombra igualmente.
#Se agrupa el conteo por la columna Tipo de la tabla Vehiculos

,Tipo_Vehiculo,Posiciones_Servidas
0,Autobus Lanzadera,5557
1,Coche de servicio,5216
2,Pushback,5068
3,Escalera,5340
4,Ambulift,4845
5,Camión de repostaje,4900
6,Tractor equipaje,5640
7,Cinta Maletas,5555


9. ¿Cuántas puertas de embarque tiene cada terminal?


In [20]:
sql("SELECT id_terminal, COUNT(DISTINCT id) as Puertas_embarque FROM gates WHERE tipo = 'Puerta' GROUP BY id_terminal")
#Hacemos un COUNT con DISTINC en cada ID para agruparlo por id_terminal y usamos la clausula WHERE para que filtre por puerta unicamente. 

,id_terminal,Puertas_embarque
0,1,27
1,2,20


10. ¿Cuántas puertas de embarque pueden dar servicio a posiciones remotas?¿Y a posiciones con finger?¿Y a ambas?


In [21]:
#Servicio a posiciones con finger.
sql("SELECT COUNT(posicion_puerta) FROM gates WHERE tipo = 'Puerta'")

,COUNT(posicion_puerta)
0,47


In [22]:
#Servicio a posiciones remotas.
sql("SELECT COUNT(posicion_remota) FROM gates WHERE tipo = 'Pista'")

,COUNT(posicion_remota)
0,56


In [9]:
#Servicio a ambas posiciones.
sql("SELECT COUNT(*) AS Ambas_Posiciones FROM gates WHERE tipo = 'Pista' OR tipo = 'Puerta'")

,Ambas_Posiciones
0,103


11. ¿Cuántos tipos de posiciones tiene cada zona operacional?


In [13]:
sql('''SELECT zone_id, COUNT(DISTINCT tipo) as Numero_posiciones_ZonaOpe, 
    CASE
	WHEN zone_id=1 then 'ZONA A'
    WHEN zone_id=2 then 'ZONA B'
    WHEN zone_id=3 THEN 'ZONA C'
    WHEN zone_id=4 THEN 'ZONA D'
    WHEN zone_id=5 THEN 'ZONA E' END AS Zona_Oper FROM posiciones GROUP BY zone_id''')
#El zone_id corresponde a cada zona operacional A,B,C,D,E respectivamente y la columna tipo contiene 4 valores distintos S, M, L y XL

,zone_id,Numero_posiciones_ZonaOpe,Zona_Oper
0,1,4,ZONA A
1,2,4,ZONA B
2,3,4,ZONA C
3,4,4,ZONA D
4,5,4,ZONA E


12. ¿Cuál es el ratio de vehículos de cada tipo con respecto a cada tipo de posición?


In [10]:
sql('''SELECT vehiculos_tipo.tipo AS Tipo_Vehiculo, posiciones_tipo.tipo AS Tipo_Posicion, 
    COUNT(*) AS Total, CAST(COUNT(*) AS FLOAT) / subconsulta.cantidad_total AS Ratio 
    FROM vehiculos JOIN zonas_operacionales ON vehiculos.id_zona = zonas_operacionales.id 
    JOIN posiciones ON zonas_operacionales.id = posiciones.zone_id JOIN (SELECT posiciones.tipo, 
    COUNT(*) AS cantidad_total FROM posiciones GROUP BY posiciones.tipo) subconsulta ON subconsulta.tipo = posiciones.tipo 
    JOIN (SELECT vehiculos.tipo FROM vehiculos GROUP BY vehiculos.tipo) vehiculos_tipo ON vehiculos_tipo.tipo = vehiculos.tipo 
    JOIN (SELECT posiciones.tipo FROM posiciones GROUP BY posiciones.tipo) posiciones_tipo ON posiciones_tipo.tipo = posiciones.tipo 
    GROUP BY vehiculos_tipo.tipo, posiciones_tipo.tipo, subconsulta.cantidad_total LIMIT 5''')

#'''En esta consulta, estamos utilizando dos subconsultas para obtener el número total de posiciones y los tipos
#de vehículos y posiciones únicos. Luego, unimos la tabla "vehiculos" con la tabla "posiciones" a través
#de la tabla "zonas_operacionales" para obtener los registros de vehículos y posiciones que se han unido.
#A continuación, utilizamos la cláusula GROUP BY para agrupar los resultados por tipo de vehículo y tipo de posición, 
#y calculamos el total de registros y el ratio utilizando las funciones COUNT() y CAST().

#El resultado de esta consulta mostrará el tipo de vehículo, el tipo de posición, el total de registros
#y el ratio de vehículos de cada tipo con respecto a cada tipo de posición.'''

,Tipo_Vehiculo,Tipo_Posicion,Total,Ratio
0,Autobus Lanzadera,STAND S,1567,15.828283
1,Autobus Lanzadera,STAND XL,1543,15.744898
2,Autobus Lanzadera,STAND M,1327,15.611765
3,Autobus Lanzadera,STAND L,1120,16.470588
4,Coche de servicio,STAND S,1480,14.949495


13. ¿Cuántos vuelos han tenido servicio de un ambulift the swiss port?

In [31]:
sql('''SELECT  vuelos.num_vuelo AS Numero_Vuelo, vehiculos.tipo AS Tipo_Vehiculo, handling.nombre AS Handling, 
    COUNT(*) AS Cantidad_Vuelos FROM vuelos JOIN servicios ON vuelos.id = servicios.id_vuelo JOIN vehiculos 
    ON servicios.id_vehiculo = vehiculos.id JOIN handling ON vehiculos.id_handling = handling.id WHERE handling.nombre = 'Swissport' 
    AND vehiculos.tipo = 'ambulift' GROUP BY vuelos.num_vuelo, vehiculos.tipo, handling.nombre''')

#'''Usamos las columnas de las siguientes tablas:
#vuelos: id
 #servicios: id_vuelo, id_vehiculo 
 #vehiculos: id, id_handling
 #handling: id 
#Usamos la clausula WHERE para filtrar por los datos que nos pide la pregunta: Ambulift y Swissport.
#La función COUNT nos sirve para contar las veces que se cumple la condición en cada vuelo.
#Finalmente agrupamos por las columnas vuelos.num_vuelos, vehiculos.tipo y handling.nombre
# '''

,Numero_Vuelo,Tipo_Vehiculo,Handling,Cantidad_Vuelos
0,WP2465,Ambulift,SWISSPORT,1
1,JC0270,Ambulift,SWISSPORT,1
2,OT9797,Ambulift,SWISSPORT,1
3,NM4988,Ambulift,SWISSPORT,1
4,TP3292,Ambulift,SWISSPORT,2
...,...,...,...,...
205,DV2199,Ambulift,SWISSPORT,1
206,NJ8996,Ambulift,SWISSPORT,2
207,FS3224,Ambulift,SWISSPORT,1
208,IV7332,Ambulift,SWISSPORT,1
